In [29]:
# import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [30]:
!ls

20181203_ADS_Final.xlsx
20181203_CreateDataRegression.ipynb
20181207_Connections.csv
20181207_DataForRegression.csv
20181207_FinalMaintenanceAndTurnstile.csv
20181213_ParameterTuningAndOutput.ipynb
20181213_RandomForestShivam-Copy1.ipynb
20181213_RandomForestShivam.ipynb
20181214_RFFeatures.csv
20181214_RFOutput.csv
Final_regression-Kenji.ipynb
Haoming_Regression (1).ipynb
subway_line_station_figure.pdf
Visualization_subwayline_subwaystation_Haoming.ipynb
xgboost_improve_Haoming.ipynb
Xgboost_model.ipynb


In [31]:
# import dataframes
df_turnstile = pd.read_excel("20181203_ADS_Final.xlsx", sheet_name = "Turnstile", parse_dates= True)
df_maintenance = pd.read_excel("20181203_ADS_Final.xlsx", sheet_name ="Maintenance", parse_dates= True)
df_dict = pd.read_csv("20181207_Connections.csv")

In [32]:
df_turnstile.head()

,Stop_id,DATE1,Entries,Year,Month,Day
0,R20,2016-08-27,73677,2016,8,27
1,R20,2016-08-28,56873,2016,8,28
2,R20,2016-09-03,77103,2016,9,3
3,R20,2016-09-04,57751,2016,9,4
4,R20,2016-09-10,93473,2016,9,10


In [33]:
df_maintenance.head()

,MID,Maintenance Text,From Date,To Date,Station ID,Station Name,Keywords,Dict,Entry: Gain,From Year,From Month,From Day,To Year,To Month,To Day
0,KU-22,"9:45 PM Fri to 5 AM Mon, Jul 6 - 9 Manhattan-b...",2018-07-06,2018-07-09,G08,Forest Hills - 71 Av,Track Replacement,F-B-F07-G08 - - ;F-B-G08-G14 - - ;R-Q-G09-...,1,2018,7,6,2018,7,9
1,KU-22,"9:45 PM Fri to 5 AM Mon, Jul 6 - 9 Manhattan-b...",2018-07-06,2018-07-09,G09,67 Av,Track Replacement,R-B-G08-G09 - - ;R-B-G09-G10 - - ;R-Q-G10-...,0,2018,7,6,2018,7,9
2,KU-22,"9:45 PM Fri to 5 AM Mon, Jul 6 - 9 Manhattan-b...",2018-07-06,2018-07-09,G10,63 Dr - Rego Park,Track Replacement,R-B-G09-G10 - - ;R-B-G10-G11 - - ;R-Q-G11-...,0,2018,7,6,2018,7,9
3,KU-22,"9:45 PM Fri to 5 AM Mon, Jul 6 - 9 Manhattan-b...",2018-07-06,2018-07-09,G11,Woodhaven Blvd,Track Replacement,R-B-G10-G11 - - ;R-B-G11-G12 - - ;R-Q-G12-...,0,2018,7,6,2018,7,9
4,KU-22,"9:45 PM Fri to 5 AM Mon, Jul 6 - 9 Manhattan-b...",2018-07-06,2018-07-09,G12,Grand Av - Newtown,Track Replacement,R-B-G11-G12 - - ;R-B-G12-G13 - - ;R-Q-G13-...,0,2018,7,6,2018,7,9


#### Create a empty dataframe with all MaintenanceStations + TurnstileDates combinations

In [34]:
dateRange1= pd.date_range(start=df_turnstile['DATE1'].min(), end=df_turnstile['DATE1'].max()-pd.DateOffset(1), freq="7D")
dateRange2= pd.date_range(start=df_turnstile['DATE1'].min()+pd.DateOffset(1), end=df_turnstile['DATE1'].max(), freq="7D")
dateRange = dateRange1.append(dateRange2)

In [35]:
df_main= pd.DataFrame()
for i in df_maintenance['Station ID'].unique():
    df_temp= pd.DataFrame()
    df_temp['Date'] = dateRange
    df_temp['Station'] = i
    df_main = df_main.append(df_temp)
    del df_temp
df_main['Weekday'] = df_main['Date'].dt.dayofweek
df_main.reset_index(inplace = True)
df_main.head()

,index,Date,Station,Weekday
0,0,2016-08-27,G08,5
1,1,2016-09-03,G08,5
2,2,2016-09-10,G08,5
3,3,2016-09-17,G08,5
4,4,2016-09-24,G08,5


#### Combine df_main with maintenance and turnstile

In [36]:
# Combine Maintenance
def check_maintenance(row):
    df_temp = df_maintenance.loc[(df_maintenance['Station ID'] == row['Station']) 
                                 & (df_maintenance['From Date']<= row['Date'])
                                 & (df_maintenance['To Date']>= row['Date']),:]
    if len(df_temp)>0:
        return len(df_temp), sum(df_temp['Entry: Gain'])
    else:
        return 0, 0
    del df_temp

df_main['Maintenance'],df_main['EntryGain'] = zip(*df_main.apply( lambda row: check_maintenance(row), axis= 1))

In [37]:
np.sum(df_main['Maintenance']>0)

1380

In [38]:
# Combine turnstile
df_main  = df_main.merge(df_turnstile, left_on=['Station','Date'], right_on=['Stop_id','DATE1'], how='left')
df_main.drop(['Stop_id', 'DATE1', 'Year', 'Month', 'Day'], inplace=True, axis=1)

#### Find the residual after removing moving average and dividing by moving standard mean

In [39]:
def movingMeanStd(row, period):
    date_range1 = pd.date_range(end = row['Date'], periods=period, freq='7D')
    date_range2 = pd.date_range(start = row['Date'], periods=period, freq='7D')
    date_range = date_range1.append(date_range2)
    df_temp = df_main.loc[(df_main['Station'] == row['Station'])& (df_main['Date'].isin(date_range)),'Entries']
    
    if len(df_temp) > period:
        if df_temp.isna().sum() < period:
            return df_temp.mean(), df_temp.std()
        else:
            return np.nan, np.nan
    else:
        return np.nan, np.nan
    

In [40]:
#### Find the optimal time window
#df_score = pd.DataFrame()
#df_tempScore = pd.DataFrame()
#for period in [2,3,4,5,6,7,8,9,10]:
#    df_main['Mean'], df_main['Std'] = zip(*df_main.apply(lambda row: movingMeanStd(row, period), axis=1))
#    df_main['Score'] = (df_main['Entries'] - df_main['Mean'])/df_main['Std']
#    df_tempScore = pd.DataFrame()
#    df_tempScore.loc[period,'MainMedian'] = df_main.loc[(df_main['Maintenance']>0)&(df_main['EntryGain']==0), 'Score'].median()
#    df_tempScore.loc[period,'MainMean'] = df_main.loc[(df_main['Maintenance']>0)&(df_main['EntryGain']==0), 'Score'].mean()
#    df_tempScore.loc[period,'GainMedian'] = df_main.loc[(df_main['Maintenance']>0)&(df_main['EntryGain']==1), 'Score'].median()
#    df_tempScore.loc[period,'GainMean'] = df_main.loc[(df_main['Maintenance']>0)&(df_main['EntryGain']==1), 'Score'].mean()
#    df_tempScore.loc[period,'NormMedian'] = df_main.loc[(df_main['Maintenance']==0)&(df_main['EntryGain']==0), 'Score'].median()
#    df_tempScore.loc[period,'NormMean'] = df_main.loc[(df_main['Maintenance']==0)&(df_main['EntryGain']==0), 'Score'].mean()
#    df_score = df_score.append(df_tempScore)
#for i in df_score.columns:
#    plt.plot(df_score.index, df_score[i], label = i)
#plt.legend()


In [41]:
# Mean, Std and Score
df_main['Mean'], df_main['Std'] = zip(*df_main.apply(lambda row: movingMeanStd(row, 5), axis=1))
df_main['Score'] = (df_main['Entries'] - df_main['Mean'])

In [42]:
df_main.head()

,index,Date,Station,Weekday,Maintenance,EntryGain,Entries,Mean,Std,Score
0,0,2016-08-27,G08,5,0,0,14346.0,NaN,NaN,NaN
1,1,2016-09-03,G08,5,0,0,14854.0,16335.833333,1451.473102,-1481.833333
2,2,2016-09-10,G08,5,0,0,16677.0,16591.714286,1487.941723,85.285714
3,3,2016-09-17,G08,5,0,0,18175.0,16709.750000,1417.444355,1465.250000
4,4,2016-09-24,G08,5,0,0,17133.0,16663.111111,1333.259769,469.888889


In [43]:
df_main['Std1'] = np.nan
for i in df_main['Station'].unique():
    for j in df_main['Weekday'].unique():
        df_main.loc[(df_main['Station']==i)&(df_main['Weekday']==j),'Std1'] = df_main.loc[(df_main['Station']==i)&(df_main['Weekday']==j),'Score'].std()   

In [44]:
df_main['Score'] = (df_main['Entries'] - df_main['Mean'])/df_main['Std1']

In [46]:
# Remove missing value
df_main = df_main.dropna()

In [98]:
df_lines = pd.read_excel("20181203_ADS_Final.xlsx", sheet_name ="Dict", parse_dates= True)
df_lines['lines']= df_lines['DICT1'].apply(lambda x: x.split('-')[0])
df_lines.drop(['DICT1','DICT2'], inplace=True, axis=1)

# replace values
df_lines.replace(to_replace='D17', value='P01', inplace=True)
df_lines.replace(to_replace='R17', value='P01', inplace=True)
df_lines.replace(to_replace='G20', value='P02', inplace=True)
df_lines.replace(to_replace='R36', value='P02', inplace=True)
df_lines.replace(to_replace='F23', value='P03', inplace=True)
df_lines.replace(to_replace='R33', value='P03', inplace=True)
df_lines.replace(to_replace='Q01', value='P04', inplace=True)
df_lines.replace(to_replace='R23', value='P04', inplace=True)

df_lines= df_lines.drop_duplicates(['stop_id','lines'])
df_lines['Count']=1


df_lines = pd.DataFrame(df_lines.pivot(index= 'stop_id', columns='lines', values='Count'))
df_lines.fillna(0, inplace=True)

In [99]:
df_main1 = df_main.merge(df_lines, left_on='Station', right_on=df_lines.index, how='left')

In [101]:
# Save output file
df_main1.to_csv("20181207_FinalMainAndTurn.csv")

#### *END*: Don't refer to anything after this point. Work in progress

In [49]:
dict1 = {'Track Replacement':'Track',
 'Station Enhancements':'Station',
 'Signal Improvements':'Signal',
 'Track Maintenance':'Track',
 'Electrical Improvements':'Electrical',
 'track replacement':'Track',
 'track maintenance':'Track',
 'Structural improvements':'Structural',
 'Signal Modernization':'Signal',
 'station enhancements':'Station',
 'signal modernization':'Signal',
 'track maintenance. ':'Track',
 'electrical improvements':'Electrical',
 'structural improvements':'Structural',
 'signal maintenance':'Signal',
 'Signal Maintenance':'Signal',
 'Station Improvements':'Station',
 'Electrical improvements':'Electrical',
 'Structural Improvements':'Structural',
 'Station Rehabilitation':'Station'}

# Category of maintenance
df_maintenance['Category'] = df_maintenance['Keywords'].map(dict1)

In [50]:
# Combine Maintenance
def maintenance_feature(row):
    dict_temp = df_maintenance.loc[(df_maintenance['Station ID'] == row['Station']) 
                                 & (df_maintenance['From Date']<= row['Date'])
                                 & (df_maintenance['To Date']>= row['Date']),:]
    dict_temp['F'] = dict_temp.apply(lambda x: 'F-' in x['Dict'], axis = 1)
    dict_temp['N'] = dict_temp.apply(lambda x: 'N-' in x['Dict'], axis = 1)
    dict_temp['R'] = dict_temp.apply(lambda x: 'R-' in x['Dict'], axis = 1)
    
    dict_temp['Track'] = dict_temp.apply(lambda x: 'Track' in x['Category'], axis = 1)
    dict_temp['StationImp'] = dict_temp.apply(lambda x: 'Station' in x['Category'], axis = 1)
    dict_temp['Signal'] = dict_temp.apply(lambda x: 'Signal' in x['Category'], axis = 1)
    dict_temp['Structural'] = dict_temp.apply(lambda x: 'Structural' in x['Category'], axis = 1)
    dict_temp['Electrical'] = dict_temp.apply(lambda x: 'Electrical' in x['Category'], axis = 1)
    
    return dict_temp['F'].sum(),dict_temp['N'].sum(),dict_temp['R'].sum(),dict_temp['Track'].sum(),dict_temp['StationImp'].sum(),dict_temp['Signal'].sum(),dict_temp['Structural'].sum(),dict_temp['Electrical'].sum()

    del dict_temp


In [51]:
df_mainSubset =df_main[df_main['Maintenance']>0]

In [52]:
df_mainSubset['F'],df_mainSubset['N'],df_mainSubset['R'],df_mainSubset['Track'],df_mainSubset['StationImp'],df_mainSubset['Signal'],df_mainSubset['Structural'],df_mainSubset['Electrical'] = zip(*df_mainSubset.apply(lambda row: maintenance_feature(row), axis=1))


In [53]:
# Combine Maintenance
def parse_dict(row):
    dict_temp = df_maintenance.loc[(df_maintenance['Station ID'] == row['Station']) 
                                 & (df_maintenance['From Date']<= row['Date'])
                                 & (df_maintenance['To Date']>= row['Date']),:]
    a=[]
    for i in range(len(dict_temp)):
        a = a + list(dict_temp['Dict'])[i].split(';')[0:len(list(dict_temp['Dict'])[i].split(';'))-1]
    series = pd.DataFrame({'Dict': a})
    series['Flag'] = series.apply(lambda x: 'blank' in x['Dict'], axis =1)
    series = series[series['Flag']==False]
    series = series[series['Dict'].apply( lambda x: len(x)>15)]
    series['Line'], series['Dirc'], series['From'], series['To'], series['Flag'], series['Shuttle'] = zip(*series.apply(lambda row: row['Dict'].split('-'), axis =1))
    series.sort_values(['Flag','Shuttle'], ascending=False, inplace =True)
    series.reset_index(inplace=True, drop=True)
    series = series.drop_duplicates(['Line', 'Dirc', 'From', 'To'])
    series['Flag']= series.apply(lambda x: int('1' in x['Flag']), axis=1)
    series['Shuttle']= series.apply(lambda x: int('1' in x['Shuttle']), axis=1)
    
    return len(series), series['Flag'].sum(), series['Shuttle'].sum()
    del dict_temp, series


In [54]:
df_mainSubset['ConnectionDay'], df_mainSubset['Broken'], df_mainSubset['Shuttle'] = zip(*df_mainSubset.apply(lambda row: parse_dict(row), axis=1))

In [55]:
# add original connection
df_mainSubset =df_mainSubset.merge(df_dict, on='Station', how= 'left')

In [56]:
# Boolean
df_mainSubset[['F', 'N', 'R', 'Track', 'StationImp','Signal', 'Structural', 'Electrical']] = df_mainSubset[['F', 'N', 'R', 'Track', 'StationImp','Signal', 'Structural', 'Electrical']].apply(lambda x: x>0)

In [57]:
df_mainSubset['NewConn'] = df_mainSubset.apply(lambda x: max(0,x['ConnectionDay']-x['OriginalConnection']), axis=1)

In [58]:
df_mainSubset['NewConnRatio'] = df_mainSubset['NewConn']/df_mainSubset['OriginalConnection']

In [59]:
df_mainSubset['LiveConn'] = df_mainSubset['OriginalConnection'] -df_mainSubset['Broken'] +df_mainSubset['NewConn']

In [ ]:
df_mainSubset['LiveConnRatio'] = df_mainSubset['LiveConn']/df_mainSubset['OriginalConnection']

In [60]:
df_mainSubset['Replacement'] = df_mainSubset.apply(lambda x: max(0,x['Broken']-x['Shuttle']), axis=1)

In [61]:
df_mainSubset['Saturday'] =df_mainSubset['Weekday'].apply(lambda x: x==5) 

In [62]:
df_mainSubset['StdRatio'] = df_mainSubset['Std']/df_mainSubset['Mean']

In [63]:
df_mainSubset.columns


Index(['index', 'Date', 'Station', 'Weekday', 'Maintenance', 'EntryGain',
       'Entries', 'Mean', 'Std', 'Score', 'Std1', 'F', 'N', 'R', 'Track',
       'StationImp', 'Signal', 'Structural', 'Electrical', 'ConnectionDay',
       'Broken', 'Shuttle', 'OriginalConnection', 'NewConn', 'NewConnRatio',
       'LiveConn', 'Replacement', 'Saturday', 'StdRatio'],
      dtype='object')

In [64]:
df_mainSubset.to_csv("20181214_FinalMaintenance.csv")

In [50]:
df_final = df_mainSubset[['NewConn', 'NewConnRatio', 'LiveConn',
       'LiveConnRatio', 'Replacement', 'Saturday','Maintenance', 'EntryGain','F', 'N', 'R', 'Track', 'StationImp',
       'Signal', 'Structural', 'Electrical','ConnectionDay', 'Broken',
       'Shuttle', 'OriginalConnection','StdRatio','Score']]

In [51]:
df_final.to_csv("20181207_DataForRegression.csv", index=False)

In [ ]:
from sklearn.metrics import r2_score
df_temp = df_main.groupby(['Maintenance','EntryGain'], as_index = False)['Score'].mean()
df_eval = df_main.merge(df_temp, on= ['Maintenance', 'EntryGain'], how ='left')
r2_score(df_eval['Score_x'], df_eval['Score_y'])

In [ ]:
r2_score(df_eval.loc[(df_eval['Maintenance']==0)&(df_eval['EntryGain']==0),'Score_x'],df_eval.loc[(df_eval['Maintenance']==0)&(df_eval['EntryGain']==0),'Score_y'])

In [ ]:
error = df_eval['Score_x'] - df_eval['Score_y']
norm = df_eval['Score_y'] - np.mean(df_eval['Score_y'])
1 - error.dot(error)/norm.dot(norm)

In [ ]:
plt.hist(df_main.loc[(df_main['Maintenance']>0)&(df_main['EntryGain']==0),'Score'])

In [ ]:
plt.hist(df_main.loc[(df_main['Maintenance']>0),'Score'])

In [ ]:
plt.hist(df_main.loc[df_main['Maintenance']==0,'Score'])

In [ ]:
df_main.loc[(df_main['Maintenance']>0)&(df_main['EntryGain']==0),'Score'].describe(),"/n" ,df_main.loc[df_main['Maintenance']==0,'Score'].describe()